# Проектная работа

# ВВЕДЕНИЕ 

Данный проект является анализом интернет-магазина . 
Задачи , которые ставятся для реализации этого проекта :
*нужно выявить определяющие успешность игры закономерности, проанализирорав рынки продажи игр, влияние рейтинга на их популярность, распределение по жанрам и платформам.


## ШАГ 2. загрузка датасета и ознакомление




### Для начала добавим библиотеки




In [1]:
pip install -U seaborn

Requirement already up-to-date: seaborn in /srv/conda/envs/notebook/lib/python3.6/site-packages (0.11.1)
Note: you may need to restart the kernel to use updated packages.


обновил  seaborn , потому что не запускался модуль seaborn.histplot

In [2]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns  
from scipy import stats as st
# импорт библиотек

### далее необходимо загрузить датасет и ознакомиться с ним

In [3]:
df = pd.read_csv('/datasets/games.csv')   # подгружаем датасет

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
df.head(10)# получение первых 10 строк таблицы df

In [ ]:
df.info()# получение общей информации о данных в таблице df

In [ ]:
df.columns  #вывод названий столбцов

# ВЫВОД
*имеются индексы с разными регистрами

*в данных есть много пропусков

*не все данные с нужным типом данных

## ШАГ 3. Подготовка данных




###  приступим к предобработке данных. 
Для начала необходимо привести все значения названий столбцов к нижнему регистру.


In [ ]:

#приводим все заголовки к нижнему регистру


df.rename(columns={'Name': 'name', 'Platform': 'platform','Year_of_Release': 'year_of_release','Genre': 'genre','NA_sales': 'na_sales', 'EU_sales': 'eu_sales', 'JP_sales': 'jp_sales', 'Other_sales': 'other_sales', 'Critic_Score': 'critic_score', 'User_Score': 'user_score', 'Rating': 'rating'}, inplace=True)

### Теперь необходимо разобраться с типами данных.

In [ ]:
df.dtypes #проверка типов данных


год выпуска игры имеет неверный формат.   дата в виде 2017.0 (с плавающей запятой)не подходит , поэтому необходимо изменить тип данных на цельночисленный

In [ ]:
# Изменим тип данных в столбце year_of_release 
df['year_of_release'] = df['year_of_release'].astype('Int64')

теперь даты в нужном формате , удобном для дальнейшей обработки.

есть вопрос с данными в user_score. в этом столбце есть  значения tbd , которые меняют тип данных , но с ним разберемся позже .       

In [ ]:
df.loc[df['user_score'] == 'tbd']     

### Далее проверка на пропуски данных


In [ ]:
df.isnull().sum() #найдем все пропуски данных в датасете

#### далее по каждомоу из пропущеных столбцов отдельно.

##### name  и genre

In [ ]:
df[df['name'].isna()]  # проверим какие значения не имеют имени

In [ ]:
df['name']= df['name'].fillna('unknown')# заменим пропущеные названия игр на unknown

пропущеных имен не осталось, как видим два пропущеных жанра , так же относятся к строкам с пропущеными именами , сразу их и исправим 

In [ ]:
df['genre']= df['genre'].fillna('unknown')# заменим пропущеные жанры на unknown

##### year_of_release     

возьмем игры с пропущеным годом выпуска и припишем им даты выпуска для этих же игр с других платформ.

In [ ]:
for i in df[df['year_of_release'].isnull() == True].index:  
    df['year_of_release'][i] = df.loc[df['name'] == df['name'][i], 'year_of_release'].max()

In [ ]:
df[df['year_of_release'].isna()] 

осталось еще 146 пропусков

##### critic_score

Разберемся с оценками критиков

In [ ]:
df[df['critic_score'].isna()]  # просмотрим пропуски в critic_score

сейчас имеется 8578 пропусков , но в данный момент эти значения не нужны . Позже при решении следующих задач на основании требований будем заполнять их.

##### user_score

In [ ]:
df[df['user_score'].isna()]  # просмотрим пропуски в user_score

встречается  аббревиатура 'tbd' в столбцах с пользовательским рейтингом .
следовательно в этих местах рейтинг еще не определен , возможно ввиду малого количества оценок

In [ ]:
df.loc[df['user_score'] == 'tbd']       

эти значения никак нам не помогут поэтому лучше их заменить на пустые поля 

In [ ]:
df['user_score'] = df['user_score'].replace('tbd', np.nan) # удалим значения tbd

In [ ]:
df.loc[df['user_score'] == 'tbd']     # удостоверимся  ,что не осталось значений tbd  

In [ ]:
df['user_score'] = df['user_score'].astype(float)    # приведем значения user_score к типу данных как и у critic_score, чтобы все рейтинги были одного формата
df['user_score'].dtype

##### rating

In [ ]:
df[df['rating'].isna()]  # просмотрим пропуски в rating

эти значения пропусков сейчас нам не особо важны . так же обратим внимание на то , что рейтинг отстутвтует вместе с оценками критиков . для этого необходимо детальнее разобраться с ними далее по ходу решения.

#### Так же необходимо удостовериться , что отсутсвуют дубликаты.

In [ ]:
df.duplicated().sum()#  суммарное количество дубликатов в таблице df

дубликаты отсутствуют , значит можно смело двигаться дальше.  никакие значения нигде не задваиваются тем самым не испортят чистоту данных.

### Посчитаем суммарные продажи во всех регионах и запишим их в отдельный столбец

In [ ]:
df['sum_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']  # добавляем новый столбец и суммируем в него все продажи по регионам

In [ ]:
df.head(10)  # результат

# ВЫВОД 

*остались пропуски в данных , но для их заполнения нужен более детальный анализ

*значения tbd удалены , так как не несли нагрузки , но не являлись пропусками (хотя таковыми являются)

*год выпуска игр по возможности заполнен имеющмимися данными этих же игр с других платформ



## Шаг 4. Проведем исследовательский анализ данных

### Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?

In [ ]:
games_on_years = df.pivot_table(index='year_of_release', values='name', aggfunc='count')  # создадим таблицу с данными , которые нужны для визуализации

In [ ]:
games_on_years.head(10)

Визуализируем данные по количеству игр за разные годы выпуска.


In [ ]:


games_on_years.plot(figsize=(17,4))

grid1 = plt.grid(True) 
plt.title("сколько игр выпускалось в разные годы")
plt.show()

на графике видно , что самыми продуктивными  были 2008 и 2009 года  , после налблюдается резкий спад , который можно свзять с множеством факторов , начиная от развития мобильных платформ , заканчивая кризисом 2008 года , как раз студии могли успеть выпустить почти готовые проекты на рынок, а новых начинать не стали.

 за все периоды рассматривать осбого смысла данные нет . до 2000 годов игры не явзялись прибыльным бизнесом да и качество игр  и бюджеты были совершенно другими. 

### Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. Найдите популярные в прошлом платформы, у которых сейчас продажи на нуле. За какой характерный период появляются новые и исчезают старые платформы?

In [ ]:
platform_sales = df.pivot_table(index='platform', values='sum_sales', aggfunc='sum').sort_values(by='sum_sales', ascending=False)
# создадим таблицу с данными , которые нужны для визуализации

#отсортируем значения по доходу от продаж

In [ ]:
platform_sales.head(10)

In [ ]:
platform_sales.plot.bar(figsize=(15,7))


plt.title("Продажи по платформам")
plt.show()

playstation явно в лидерах , особенно если сложить 2 и 3 поколение консоли отрыв будет недосягаемым.

Возьмем топ 6 платформ ,так как PS прдеставлена в топе в 3 поколениях и вытесняет всех окончательно.
плюс отрыв PS от gba больше чем в два раза и это существенный фактор для отделения следующей платформы от лидеров продаж.

In [ ]:
top6_platform=df.pivot_table(index=['platform',], values='sum_sales', aggfunc='sum').sort_values(by='sum_sales', ascending=False).head(6)

In [ ]:
top6_platform = top6_platform.reset_index().rename_axis(None, axis=1)
top6_platform


Теперь необходимо построить распределение по годам.

In [ ]:
def sale_year_platform(name, data):
    platform_release = data[(data['platform'] == name) & (data['year_of_release'] > 2000)]
    total = platform_release.pivot_table(index='year_of_release', values='sum_sales', aggfunc='sum').sort_values('year_of_release', ascending=False)
    return total    # напишем функцию , которая будет приводить к году выпуска количество игр на платформе

Визуализируем полученные данные

In [ ]:
plt.figure(figsize=(17,4))
plt.title('Количество продаж популярных игровых платформ')
plt.xlabel('Годы игровых релизов')
plt.ylabel('Продажи')

for i in list(top6_platform['platform']):
    sns.lineplot(data=sale_year_platform(i,df)['sum_sales'], label=i)
    plt.legend()

в среднем платформа существует от 7 до 10 лет.
все платформы преимущественно выходят поколениями , то есть в одно время сразу несколько и примерно за то же время они угасают , давая место новым поколениям.
На примере PS 2 и PS 3 можно заметить , что выход новой консоли был на спад.

в целом пик продаж приходится на 4-5 лет после выхода консоли.

### Определите, данные за какой период нужно взять, чтобы исключить значимое искажение распределения по платформам в 2016 году.

### 
с 2000 по 2016 год график выглядит как нормальное распределение , с минимальными отклонениями, но если необходимо сделать анализ исходя из значений за 2016 год, лучше брать период в 1 или 2 года , соответсвенно 2015-2016гг.

### Далее работайте только с данными, которые вы определили. Не учитывайте данные за предыдущие годы.

так как данных за 1-2 года недостаточно, ввиду имеющихся пропусков , нерелевантности некоторых значений , поскольку еще не все данные могли быть получены (в 2016 году иметь все данные за 2016 год невозможно) , увеличим рассматриваемый период до 4 лет.
так же период увеличим, что бы можно было поймать полупериод существование платформы.

In [ ]:
df_act = df[df['year_of_release']>2012]

In [ ]:
df_act

### Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.


выберем потенциально прибыльную платформу

In [ ]:
top6_platform_act=df_act.pivot_table(index=['platform',], values='sum_sales', aggfunc='sum').sort_values(by='sum_sales', ascending=False).head(6)

top6_platform_act


In [ ]:
top6_platform

как мы видимо из самых успешных платформ за все время за последние 3 года ниодна не попадает в топ 6 по продажам .


лидером до сих пор является PlayStation , но уже 4 поколение консоли, все еще приносит доход и 3 поколение .
следом идет Xbox One, кторый является 2 поколением X360 .





поэтому можно смело утверждать , что PS4 и Xone являются потенциально прибыльными платформами.
это идейные наследники предыдущих успешных платофрм , которые уже являются лидерами по продажам в настоящее время.

### Постройте график «ящик с усами» по глобальным продажам каждой игры и разбивкой по платформам. Велика ли разница в продажах? А в средних продажах на разных платформах? Опишите результат.

In [ ]:
#Сохраним в переменной  топ 6 платформ 
top_6_platforms = ['PS4','PS3','XOne','3DS','X360','WiiU']

In [ ]:

df_top_6_platforms = df[df['platform'].isin(top_6_platforms)]   # сделаем выборку по необходимым платформам


In [ ]:
df_top_6_platforms['sum_sales'].describe()    # рассмотрим основные параметры полученных данных

In [ ]:
df_top_6_platforms = df_top_6_platforms[df_top_6_platforms['sum_sales']<1.2]
# избавимся от значений выбивающихся от средних , т.к. среднеквадратическое отклонение 1.7  уберем все значение более 1.8


Построим график «ящик с усами»

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(data=df_top_6_platforms, x='platform', y='sum_sales')

plt.title('Ящик с усами', fontsize=11)
plt.xlabel('Платформа', fontsize=11)
plt.ylabel('суммарные продажи',fontsize=11)

# ВЫВОД
    Если не исключать выбросы значений , то наглядный график построить не удастся, так-как нижний и верхний квартиль ,        вместе с медианой будут одной полоской на графике 
    Исходя из графиков видно, что медиана протекает у всех по разному
    Наблюдаемые максимумы у всех платформ так же отличаются
    Лидирует по продажам  Wii и PS3 и X360, следом идут  PS4 и  Xone,  и 3DS на последнем месте. 


    Разница между платфомами обусловлена многими факторами. Начиная от региона , где они лидируют , соответсвенно и охвата аудитории , так же и тем , что эти платформы существовали в разные годы , а следовательно и конкуренция была разная.
    
    
    PS3 и X360 старые платформы, которые еще имеют больший объем на рынке , чем их более свежие поколения , которые не   успели разойтись достаточным тиражом.
    WiiU так же довольно старая платформа, релиз которой состоялся еще в 2012 году , и имея достаточное количество пользователей , ей это позволяет держаться в лидерах в этом показателе.

### Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы и соотнесите их с продажами игр на других платформах.

#### Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. 

In [ ]:
PS4 = df_act[df_act['platform']=='PS4'] #создадим переменную , в который будем держать все данные о платформе PS4

In [ ]:
PS4

построим диаграммы рассеяния для сравнения рейтинга пользователей и критиков и посчтитаем корреляцию рейтинга к продажам

для начала для оценок пользователей

In [ ]:
#Корреляция между оценкой пользователей и суммарными продажами
PS4['user_score'].corr(PS4['sum_sales'])

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=PS4)

plt.title('диаграмма рассеяния для рейтинга пользователей')

и для оценок критиков

In [ ]:
#Корреляция между оценкой критиков и суммарными продажами
PS4['critic_score'].corr(PS4['sum_sales'])

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=PS4)
plt.title('диаграмма рассеяния для рейтинга критиков')

#### Сформулируйте выводы

# ВЫВОД
Корреляция между оценкой пользователей и суммарными продажами на  PS4  отрицательная  -0.031957110204556424

Корреляция между оценкой критиков и суммарными продажами на  PS4  Умеренная 0.40656790206178095

#### соотнесите выводы с продажами игр на других платформах.

##### Xone

In [ ]:
Xone = df_act[df_act['platform']=='XOne']

In [ ]:
Xone['user_score'].corr(Xone['sum_sales'])

In [ ]:
Xone['critic_score'].corr(Xone['sum_sales'])

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=Xone)
plt.title('диаграмма рассеяния для рейтинга критиков')

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=Xone)

plt.title('диаграмма рассеяния для рейтинга пользователей')

##### PS3

In [ ]:
ps3 = df_act[df_act['platform']=='PS3']

In [ ]:
ps3['user_score'].corr(ps3['sum_sales'])

In [ ]:
ps3['critic_score'].corr(ps3['sum_sales'])

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=ps3)
plt.title('диаграмма рассеяния для рейтинга критиков')

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=ps3)

plt.title('диаграмма рассеяния для рейтинга пользователей')

##### 3DS

In [ ]:
DS = df_act[df_act['platform']=='3DS']

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=DS)

plt.title('диаграмма рассеяния для рейтинга пользователей')

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=DS)
plt.title('диаграмма рассеяния для рейтинга критиков')

In [ ]:
DS['user_score'].corr(DS['sum_sales'])

In [ ]:
DS['critic_score'].corr(DS['sum_sales'])

##### X360

In [ ]:
X360 = df_act[df_act['platform']=='X360']

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=X360)

plt.title('диаграмма рассеяния для рейтинга пользователей')

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=X360)
plt.title('диаграмма рассеяния для рейтинга критиков')

In [ ]:
X360['user_score'].corr(X360['sum_sales'])

In [ ]:
X360['critic_score'].corr(X360['sum_sales'])

##### WiiU

In [ ]:
WiiU = df_act[df_act['platform']=='WiiU']

In [ ]:
#Построим диаграмму рассеяния  user_score
plt.figure(figsize=(15,4))
sns.scatterplot(x='user_score', y='sum_sales', data=WiiU)

plt.title('диаграмма рассеяния для рейтинга пользователей')

In [ ]:
#Построим диаграмму рассеяния  critic_score
plt.figure(figsize=(15,4))

sns.scatterplot(x='critic_score', y='sum_sales', data=WiiU)
plt.title('диаграмма рассеяния для рейтинга критиков')

In [ ]:
WiiU['user_score'].corr(WiiU['sum_sales'])

In [ ]:
WiiU['critic_score'].corr(WiiU['sum_sales'])

# ВЫВОД


Корреляция между оценкой пользователей и суммарными продажами на PS4 отрицательная -0.031957110204556424

Корреляция между оценкой критиков и суммарными продажами на PS4 Умеренная 0.40656790206178095

________________________________________________________________________________________________________


Корреляция между оценкой пользователей и суммарными продажами на  Xbox One  отрицательная -0.06892505328279414

Корреляция между оценкой критиков и суммарными продажами на  Xbox One  Умеренная 0.4169983280084017

________________________________________________________________________________________________________



Корреляция между оценкой пользователей и суммарными продажами на  PS3  Слабая 0.0023944027357566925

Корреляция между оценкой критиков и суммарными продажами на  PS3  Умеренная 0.3342853393371919


________________________________________________________________________________________________________



Корреляция между оценкой пользователей и суммарными продажами на  3DS  Слабая 0.24150411773563016

Корреляция между оценкой критиков и суммарными продажами на  3DS  Умеренная 0.3570566142288103


________________________________________________________________________________________________________



Корреляция между оценкой пользователей и суммарными продажами на X360 отрицательная -0.011742185147181342

Корреляция между оценкой критиков и суммарными продажами на X360 Умеренная 0.3503445460228664



________________________________________________________________________________________________________


Корреляция между оценкой пользователей и суммарными продажами на WiiU Умеренная 0.4193304819266187

Корреляция между оценкой критиков и суммарными продажами на WiiU Умеренная 0.3764149065423912

в большинстве случаев влияние оценки пользователей на продажи очень слабое и брать ее в расчет нет смысла.
исключением является WiiU, на которйы влияние пользовательского рейтинга сильнее чем оценки критиков , возможно это из-за того , что у WiiU очень дружное коммьюнити , которое предпочитает больше брать информацию с местных формумов , а не доверять на слово изданиям.

Оценка критиков же наоборот на всех платформах имеет вес .


### Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
genres_cnt = df_act.pivot_table(
    index='genre', values='sum_sales', aggfunc='count').sort_values(by='sum_sales', ascending=False)
genres_cnt     # рассмотрим распределение по количеству игр в каждом жанре

In [ ]:
genres_cnt.plot.bar(figsize=(15,4))
plt.xlabel('Жанры игр',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('Распределение игр по жанрам ',fontsize=10)
plt.show()     # визуализируем данные 

больше всего игр Action ,меньше всего пазлов и стратегий

In [ ]:
# рассмотрим распределение по доходности каждого жанра
genres_top = df.pivot_table(
    index='genre', values='sum_sales', aggfunc='sum').sort_values(by='sum_sales', ascending=False)

genres_top

In [ ]:
genres_top.plot.bar(figsize=(15,4))
plt.xlabel('Жанры игр',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('Распределение игр по жанрам ',fontsize=10)
plt.show()     # визуализируем данные 

самые прибыльне игры это Action,  Strategy же приносят меншье всего дохода

в целом есть прямая зависимость от того , сколько было всего выпущено игр в жанре и сколько жанр принес дохода .
так самый массовый жанр экшнов и спортивные игры принесли больше всего дохода.
меньше всего прибыли от пазлов и стратегий - их в свою очередь было выпущено меньше всего.

## Шаг 5. Составьте портрет пользователя каждого региона

### Определите для пользователя каждого региона (NA, EU, JP):

#### Самые популярные платформы (топ-5). Опишите различия в долях продаж.

##### NA


In [ ]:
na_platform_act = df_act.pivot_table(index='platform', values='na_sales', aggfunc='sum').sort_values(by='na_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных платформах в NA на актуальное время

In [ ]:
na_platform_act

In [ ]:
na_platform_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные платформы в NA ',fontsize=10)
plt.show()     # визуализируем данные 

In [ ]:
na_platform = df.pivot_table(index='platform', values='na_sales', aggfunc='sum').sort_values(by='na_sales', ascending=False).head(3) 
# создадим таблицу с данными о  3 самых популярных платформах в NA

In [ ]:
na_platform

по актуальным данным 
В США самая популярная платформа -PS4 ,следом XOne и X360
за весь период же самая популярная платформа в США это X360

##### EU

In [ ]:
eu_platform_act = df_act.pivot_table(index='platform', values='eu_sales', aggfunc='sum').sort_values(by='eu_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных платформах в EU на актуальное время

In [ ]:
eu_platform_act

In [ ]:
eu_platform_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные платформы в EU ',fontsize=10)
plt.show()     # визуализируем данные 

In [ ]:
eu_platform = df.pivot_table(index='platform', values='eu_sales', aggfunc='sum').sort_values(by='eu_sales', ascending=False).head(3) 
# создадим таблицу с данными о  3 самых популярных платформах в EU 

In [ ]:
eu_platform

### В европе лидерство у PS4 и PS3 следом идет xbox 
безоговорочное лидерство занимает PS 4 за последние 3 года.
за все время самая популярная консоль в европе это PS2


##### JP

In [ ]:
jp_platform_act = df_act.pivot_table(index='platform', values='jp_sales', aggfunc='sum').sort_values(by='jp_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных платформах в JP за актуальное время

In [ ]:
jp_platform_act

In [ ]:
jp_platform_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные платформы в JP ',fontsize=10)
plt.show()     # визуализируем данные 

In [ ]:
jp_platform = df.pivot_table(index='platform', values='jp_sales', aggfunc='sum').sort_values(by='jp_sales', ascending=False).head(3) 
# создадим таблицу с данными о  3 самых популярных платформах в JP 

In [ ]:
jp_platform

### в японии лидер nintendo 3DS следом PS3 PS vita и PS4 с WiiU 
xbox отсутствует ввиду популярности локальных платформ

за все время самая популярная платформа Японии это DS

# ВЫВОД
распределение по платформам прямо зависит от рынка 
на территории США хоть и не сильно, но доминирует американская консоль( по суммарным продажам Xone и x360) 
на территории Японии доминируют Японские консоли
Европа  же больше отдает предпочтение PS.

#### Самые популярные жанры (топ-5). Поясните разницу.

##### NA

In [ ]:
na_genre_act = df_act.pivot_table(index='genre', values='na_sales', aggfunc='sum').sort_values(by='na_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных жанрах в NA

In [ ]:
na_genre_act

In [ ]:
na_genre_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные жанры в NA ',fontsize=10)
plt.show()     # визуализируем данные 

лидирующие жанры в США это экшн и шутеры

##### EU

In [ ]:
eu_genre_act = df_act.pivot_table(index='genre', values='eu_sales', aggfunc='sum').sort_values(by='eu_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных жанрах в EU

In [ ]:
eu_genre_act

In [ ]:
eu_genre_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные жанры в EU',fontsize=10)
plt.show()     # визуализируем данные 

в европе жанры аналогичны США , кроме 4 позиции 
в европе вместо платформеров предпочитают гонки

##### JP

In [ ]:
jp_genre_act = df_act.pivot_table(index='genre', values='jp_sales', aggfunc='sum').sort_values(by='jp_sales', ascending=False).head(5) 
# создадим таблицу с данными о  5 самых популярных жанрах в JP

In [ ]:
jp_genre_act

In [ ]:
jp_genre_act.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('самые популярные жанры в JP',fontsize=10)
plt.show()     # визуализируем данные 

в Японии РПГ лидируют и экшн игры.
далее идут Miscellaneous файтинги и шутеры.

# ВЫВОД
рынок США и Европы аналогичны. позиция гонок в европе возможно обусловлена историческим фактором , например из-за гонки леман24,дакар,гран-при европы и.т.д и в целом развития автомобильной культуры.

распределение жанров на рынке Японии обусловлено иным культурым наследием и в целом направленностью интересов японцев на аниме сегмент, где лидирую final fantasy и др. поэтому тут превалируют РПГ а так же файтинги и Miscellaneous ,ярким представителем который является metal gear solid .

#### Влияет ли рейтинг ESRB на продажи в отдельном регионе?

##### NA

In [ ]:
na_rating = df_act.pivot_table(index='rating', values='na_sales', aggfunc='sum').sort_values(by='na_sales', ascending=False) 
# создадим таблицу с данными о  рейтинге в NA

In [ ]:

na_rating

In [ ]:
na_rating.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('рейтинг ESRB в NA',fontsize=10)
plt.show()     # визуализируем данные 

##### EU

In [ ]:
eu_rating = df_act.pivot_table(index='rating', values='eu_sales', aggfunc='sum').sort_values(by='eu_sales', ascending=False).head(6)
# создадим таблицу с данными о  рейтинге в EU

In [ ]:
eu_rating


In [ ]:
eu_rating.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('рейтинг ESRB в EU',fontsize=10)
plt.show()     # визуализируем данные 

##### JP

In [ ]:
jp_rating = df_act.pivot_table(index='rating', values='jp_sales', aggfunc='sum').sort_values(by='jp_sales', ascending=False).head(6)
# создадим таблицу с данными о  рейтинге в JP

In [ ]:
jp_rating

In [ ]:
jp_rating.plot.bar(figsize=(12,3))
plt.xlabel('платформа',fontsize=12)
plt.ylabel('Продажи',fontsize=12)
plt.title('рейтинг ESRB в JP',fontsize=10)
plt.show()     # визуализируем данные 

# ВЫВОД

рейтинг распределен в целом приблизительно одинаково в США и европе и в топе игры с рейтингом mature 17+
в японии же лидируют игры «E» («Everyone») — «Для всех» ,ввиду жесткой цензуры .

# ВЫВОД ПО ТЕМЕ 
Самые популярные игровые платформы в северной америке это: XBOX, PS2, WII, PS3, DS 
Топ 5 жанров в северной америке : action, sports, shooter, platform, misc 


Самые популярные игровые платформы в Европе это: PS2, PS3, XBOX, WII, PS 
Топ 5 жанров в Европе : action, sports, shooter, racing, misc 


Самые популярные игровые платформы в Японии это: DS, PS, PS2, SNES, 3DS 
Топ 5 жанров в Японии : Role-Playing, action, sports, platform, misc 


распределение рейтинга особо не влияет на рынок 

## Шаг 6. Проведите исследование статистических показателей

 Как изменяется пользовательский рейтинг и рейтинг критиков в различных жанрах? Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения

### Как изменяется пользовательский рейтинг и рейтинг критиков в различных жанрах? Посчитайте среднее количество, дисперсию и стандартное отклонение.

#### среднее значение

возьмем значения 4 жанров : экшн, спортивные игры, шутеры и РПГ .
выбор 4 жанров обусловлен тем , что это 4 самые массовы и в то же время самые прибыльные жанры.

жанр музыкальных игр и платформеров и прочие брать не будем , поскольку массовость/дожодность у них уже не так сильно коррелируются и в целом они ниже в списке по убыванию значений.


In [ ]:
# Код ревьюера
df_act.query("genre in ['Action', 'Sports', 'Shooter', 'Role-Playing']")\
            .groupby('genre')[['critic_score', 'user_score']]\
                    .agg(['mean', 'std', 'var'])

### Постройте гистограммы.

распределение средних пользовательских оценок 

In [ ]:
action_hist = df_act[df_act['genre']=='Action']
sport_hist = df_act[df_act['genre']=='Sports']
shooter_hist = df_act[df_act['genre']=='Shooter']
rpg_hist = df_act[df_act['genre']=='Role-Playing']
#добавим отдельные значение для каждого жанра

In [ ]:
#Построим гистограмму user_score action
plt.figure(figsize=(15,5))
sns.histplot(data=action_hist, x="user_score",color='peru')
sns.histplot(data=sport_hist, x="user_score",color='brown')
sns.histplot(data=shooter_hist, x="user_score",color='black')
sns.histplot(data=rpg_hist, x="user_score",color='green')
plt.title('гистограмма рейтинга пользователей')
plt.legend( ['action', 'sport','shooter','rpg'])


цветом выделен жанр экшн, так ак его оценки весомо отличаются от остальной массы .
но основное отличие не в самих оценках а в их количестве , так как жанр самый массовый.

распределение средних  оценок критиков 

In [ ]:
#Построим гистограмму сritic_score action
plt.figure(figsize=(15,5))
sns.histplot(data=action_hist, x="critic_score",color='peru')
sns.histplot(data=sport_hist, x="critic_score",color='green')
sns.histplot(data=shooter_hist, x="critic_score",color='brown')
sns.histplot(data=rpg_hist, x="critic_score",color='black')
plt.title('гистограмма для рейтинга критиков')
plt.legend( ['action', 'sport','shooter','rpg'])

цветом выделен жанр экшн, так ак его оценки весомо отличаются от остальной массы .
но основное отличие не в самих оценках а в их количестве , так как жанр самый массовый.

### Опишите распределения

In [ ]:
math_table

# ВЫВОД

средние оценки критиков по средним значениям колеблются в пределах погрешности .
Дисперсия у оценок критиков очень сильно варируется от жанра к жанру ,  наибольший разброс оценок наблюдается у спортивных игр, что подтверждают значения среднего отклонения , явно выделяющиеся так же среди спортивных игр.Значит среди спортивных игр встречаются как очень плохие так и очень хорошие , среди остальных игр же оценки ближе к средней величине .

рассматривая пользовательский рейтинг мы видим примернь похожую картину , что среди спортивных игр значения дисперсии и среднего отклонения выше , но уже не на столько значительно .
исходя из среднего рейтинга в 5.2 можно сделать вывод , что среди спортивных игр все же больше плохих игр .


остальные жанры в целом держатся по рейтингу близко к среднему ,значит ужасных или гениальных проектов не особо много.

## Шаг 7. Проверьте гипотезы

### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;

формулируем нулевые гипотезы:

Нулевая гипотеза Н0: Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
Альтернативная гипотеза H1: Средние пользовательские рейтинги платформ Xbox One и PC различаются

In [ ]:
xone_rel = df[(df['platform']=='XOne') & (df['year_of_release']>2000)]['year_of_release']  # посмотрим когда начались продажи xone

In [ ]:
xone_rel.hist()
plt.title('годы выпуска игр на XOne')
plt.xlabel('Годы игровых релизов')
plt.ylabel('Продажи')

мы видим что продажи для xone начинаются с 2013 года , поэтому проверять гипотезу будем в период с 2013 по 2016 год.

In [ ]:
# Сохраним в переменных необходимые для расчетов данные (актуальные данные за последние 3 года,поскольку продажи для xone начались с 2013 )
xone_ratio = df[(df['platform']=='XOne') & (df['year_of_release']>2013)]['user_score']
pc_ratio = df[(df['platform']=='PC') & (df['year_of_release']>2013)]['user_score']

In [ ]:


# средний рейтинг для xbox one
xone_ratio.mean()

In [ ]:
#средний рейтинг для PC
pc_ratio.mean()

In [ ]:
np.var(pc_ratio)

In [ ]:
np.var(xone_ratio)

In [ ]:
#Выполним проверку гипотезы. Будем использовать метод ttest_ind
xone_ratio_work=xone_ratio.dropna()   # удалим пропуски в данных , что бы они не мешали расчетам
pc_ratio_work=pc_ratio.dropna()       # удалим пропуски в данных , что бы они не мешали расчетам

alpha = .05
#equal_var=False так как дисперсии выборок не близки
results = st.ttest_ind(xone_ratio_work, pc_ratio_work,equal_var=False)#Чтобы проверить гипотезу о равенстве среднего двух генеральных
                                                           #совокупностей  по взятым из них выборкам, примените метод 
                                                           #scipy.stats.ttest_ind  (array1, array2, equal_var) .   

print('p-значение:', results.pvalue)


if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

### Вывод
Значение p-value равно  13.9%. Таким образом, не получилось опровергнуть Нулевую гипотезу. То есть, с вероятностью в 14% (округлим) мы имеем разницу в рейтингах между персональным компьютером и xbox one.

### Средние пользовательские рейтинги жанров Action (англ. «действие») и Sports (англ. «виды спорта») разные.

Сформулируем нулевые гипотезы:

Нулевая гипотеза H0: Средние пользовательские рейтинги жанров Action и Sports одинаковые
Альтернативная гипотеза H1: Средние пользовательские рейтинги жанров Action и Sports различаются

In [ ]:
gena = df[(df['genre']=='Action') & (df['year_of_release']>1980)]['year_of_release']# посмотрим когда начались продажи экшн игр
gens = df[(df['genre']=='Sports') & (df['year_of_release']>1980)]['year_of_release']# посмотрим когда начались продажи спортивных игр

In [ ]:
gena.hist()
gens.hist()
plt.title('годы выпуска спортивных игр и Экшн игр')
plt.xlabel('Годы игровых релизов')
plt.ylabel('Продажи')
plt.legend( ['action', 'sport'])


как мы видим -оба жанра существуют довольно давно , поэтому возьмем период , когда оба жанра вышли на одинаковый объем продаж с 2002 года

In [ ]:
# Сохраним в переменных необходимые для расчетов данные (актуальные данные за последние 5 лет)
genre_action = df[(df['genre']=='Action') & (df['year_of_release']>2002)]['user_score']
genre_sports = df[(df['genre']=='Sports') & (df['year_of_release']>2002)]['user_score']



In [ ]:
#средняя оценка по экшн жанру
genre_action.mean()

In [ ]:
#средняя оценка по жанру спорт
genre_sports.mean()

In [ ]:
#Выполним проверку гипотезы. Будем использовать метод ttest_ind
genre_action_work=genre_action.dropna()   # удалим пропуски в данных , что бы они не мешали расчетам
genre_sports_work=genre_sports.dropna()   # удалим пропуски в данных , что бы они не мешали расчетам

In [ ]:
np.var(genre_action_work)

In [ ]:
np.var(genre_sports_work)

In [ ]:

alpha = .05

#equal_var=False так как дисперсии выборок не близки
results = st.ttest_ind(genre_action_work, genre_sports_work, equal_var=False)   #Чтобы проверить гипотезу о равенстве среднего двух генеральных
                                                                               #совокупностей  по взятым из них выборкам, примените метод 
                                                                               #scipy.stats.ttest_ind  (array1, array2, equal_var) .   


print('p-значение:', results.pvalue)


if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

# Вывод
Значение p-value равно  0.02%. Таким образом, Отвергаем нулевую гипотезу. То есть, с околонулевой вероятностью возникнет ситуация , когда рейтинги спортивных игр будут такими же как у экшн игр.

### Задайте самостоятельно пороговое значение alpha.
Поясните:
Как вы сформулировали нулевую и альтернативную гипотезы;
Какой критерий применили для проверки гипотез и почему.

нулевая гипотеза формируется исходя из формулировки задачи с условием , что две совокупности должны быть равны ,альтернатвная гипотеза же должна быть полной противоположностью выдвинутой нулевой гипотезе.

alpha критический уровень статистической значимости.
я  взял значение равное 5% , поскольку техническая погрешность с запасом дает возможность оценить правдивость гипотезы.
поскольку нужно рассматривать равенство значений , можно было бы взять и 1% , что бы исключить возможность получения неточных данных, но разброс по рейтингу не такой большой.


## Шаг 8. Напишите общий вывод

# Общий вывод
перед тем как анализировать данные была произведена предобработка: выявлены пропуски , проверены на отсутсвтие дубликатов  ,все данные были преведены к нужным типам данных, названия столбцов приведены к нижнему регистру.
TBD значения определили как бесполезные для дальнейших расчетов и заменили их на пустые ячейки.
Год выпуска игр по возможности заполнен имеющмимися данными этих же игр с других платформ.
____________________________________________________________________________________________________________________________

Самыми продуктивными были 2008 и 2009 года , после налблюдается резкий спад , который можно свзять с множеством факторов , начиная от развития мобильных платформ , заканчивая кризисом 2008 года , как раз студии могли успеть выпустить почти готовые проекты на рынок, а новых начинать не стали.

____________________________________________________________________________________________________________________________
в среднем игровая платформа существует от 7 до 10 лет. все платформы преимущественно выходят поколениями , то есть в одно время сразу несколько и примерно за то же время они угасают , давая место новым поколениям. На примере PS 2 и PS 3 можно заметить , что выход новой консоли был на спад.

в целом пик продаж приходится на 4-5 лет после выхода консоли.
____________________________________________________________________________________________________________________________


За весь период консольных приставок самые популярные оказались: PS2, X360, PS3, WII, DS, PS

____________________________________________________________________________________________________________________________

 можно смело утверждать , что PS4 и Xone являются потенциально прибыльными платформами.
это идейные наследники предыдущих успешных платофрм , которые уже являются лидерами по продажам в настоящее время.
____________________________________________________________________________________________________________________________
в большинстве случаев влияние оценки пользователей на продажи очень слабое и брать ее в расчет нет смысла.
исключением является WiiU, на которйы влияние пользовательского рейтинга сильнее чем оценки критиков , возможно это из-за того , что у WiiU очень дружное коммьюнити , которое предпочитает больше брать информацию с местных формумов , а не доверять на слово изданиям.

Оценка критиков же наоборот на всех платформах имеет вес .

____________________________________________________________________________________________________________________________

самые прибыльне игры это Action, Strategy же приносят меншье всего дохода

в целом есть прямая зависимость от того , сколько было всего выпущено игр в жанре и сколько жанр принес дохода . так самый массовый жанр экшнов и спортивные игры принесли больше всего дохода. меньше всего прибыли от пазлов и стратегий - их в свою очередь было выпущено меньше всего.

____________________________________________________________________________________________________________________________



Самые популярные игровые платформы в северной америке это: PS4, Xone, X360, PS3, 3DS Топ 5 жанров в северной америке : action, shooter, sports, Role-Playing, misc

Самые популярные игровые платформы в Европе это: PS4, PS3, Xone, X360, 3DS Топ 5 жанров в Европе : action, shooter, sports, Role-Playing, rasing

Самые популярные игровые платформы в Японии это: 3DS, PS3, PSV, PS4, WiiU Топ 5 жанров в Японии : Role-Playing, action, misc, fighting, shooter

рейтинг распределен в целом приблизительно одинаково в США и европе и в топе игры с рейтингом mature 17+
в японии же лидируют игры «E» («Everyone») — «Для всех» ,ввиду жесткой цензуры .

____________________________________________________________________________________________________________________________
так же проверили две гипотезы 
С околонулевой вероятностью возникнет ситуация , когда рейтинги спортивных игр будут такими же как у экшн игр.
С вероятностью в 14% (округлим) мы имеем разницу в рейтингах между персональным компьютером и xbox one.
____________________________________________________________________________________________________________________________
средние оценки критиков по средним значениям колеблются в пределах погрешности .
Дисперсия у оценок критиков очень сильно варируется от жанра к жанру ,  наибольший разброс оценок наблюдается у спортивных игр, что подтверждают значения среднего отклонения , явно выделяющиеся так же среди спортивных игр.Значит среди спортивных игр встречаются как очень плохие так и очень хорошие , среди остальных игр же оценки ближе к средней величине .

рассматривая пользовательский рейтинг мы видим примернь похожую картину , что среди спортивных игр значения дисперсии и среднего отклонения выше , но уже не на столько значительно .
исходя из среднего рейтинга в 5.2 можно сделать вывод , что среди спортивных игр все же больше плохих игр .


остальные жанры в целом держатся по рейтингу близко к среднему ,значит ужасных или гениальных проектов не особо много.

____________________________________________________________________________________________________________________________

По итогу самые популярные игры это экшн игры  с рейтингом E / mature 17+  (исходя из целевого региона) .
платформа для выпуска игр предопчтительна PS4/xbox one 
другие платформы уже прекратили свои продажи либо имеют малую аудиторию и их нет смысла брать в расчет .






